## Import

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
from functools import reduce

import numpy as np
import pandas as pd
import re, time
import requests
import pickle
import warnings

warnings.filterwarnings(action='ignore')

In [2]:
import os
os.getcwd()

'C:\\Users\\PC0\\Documents\\GitHub\\Stock-price-prediction\\ipynb\\crawling-code'

In [3]:
out_data_path = 'C:\\Users\\PC0\\Documents\\datasets\\stock_price\\'
data_path = 'C:\\Users\\PC0\\Documents\\GitHub\\Stock-price-prediction\\datasets\\'

* crawling

In [4]:
options = webdriver.ChromeOptions()

options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36")

driver = webdriver.Chrome("C:\\Users\\PC0\\Desktop\\chromedriver.exe", options=options)

* metal

In [5]:
start_day = "2012/01/01"

In [6]:
urls = ['https://kr.investing.com/commodities/metals']

In [7]:
url = urls[0]

In [8]:
driver.get(url)
time.sleep(3)

In [9]:
temp_names = driver.find_elements_by_css_selector("table#cross_rate_1 tbody tr td.bold.left.noWrap.elp.plusIconTd a")
temp_flag = driver.find_elements_by_css_selector("table#cross_rate_1 tbody tr td.flag span")

In [10]:
flags = [x.get_attribute("title") for x in temp_flag]

In [11]:
names = [x + "_" + y.text if len(x) != 0 else y.text for x,y in zip(flags, temp_names)]

In [12]:
urls = [x.get_attribute("href") for x in temp_names]

In [13]:
index_url = [x + "-historical-data" if "?" not in x else x.split("?")[0] + "-historical-data?" + x.split("?")[1]for x in urls]
index_names = [x.split("-historical-data")[0].split("/")[-1] for x in index_url]

In [14]:
metal_df = pd.DataFrame({"symbol":index_names,
                         "name":names,
                         "url":index_url})

In [15]:
metal_df.to_csv(data_path + "metal_symbol_df.csv", index = False)

* loop crawling

In [16]:
index_list = []

In [17]:
start_idx = 0

In [18]:
for idx, url in tqdm(enumerate(index_url[start_idx:])):
    driver.get(url)
    time.sleep(5)

    # 날짜 입력란
    tmp_date = driver.find_element_by_css_selector("div#widgetFieldDateRange")
    tmp_date.click()
    driver.implicitly_wait(3)

    tmp_input = driver.find_element_by_css_selector("input#startDate")
    time.sleep(0.5)

    # 기존 날짜 지우기
    for _ in range(11) :
        tmp_input.send_keys(Keys.BACKSPACE)
        tmp_input = driver.find_element_by_css_selector("input#startDate")
        time.sleep(0.5)
        
    tmp_input.send_keys(start_day)

    # 요청 버튼
    request_btn = driver.find_element_by_css_selector("a#applyBtn")
    request_btn.click()
    time.sleep(5)
    
    tables = driver.find_elements_by_css_selector("table#curr_table")

    values = tables[0].text.split("\n")[1:]

    values = [re.sub("[^0-9a-zA-Z.\\- ]","",x) for x in values]

    name = names[idx]
    
    index_dict = {"date":[],
                 "{}_close".format(name):[],
                 "{}_vol".format(name):[],
                 "{}_var".format(name):[]}

    for val in values:
        year, month, day, close, _, _, _, volume, variation= val.split(" ")
        date = "-".join([year, month, day])
        index_dict['date'].append(date)
        index_dict['{}_close'.format(name)].append(close)
        index_dict['{}_vol'.format(name)].append(volume)
        index_dict['{}_var'.format(name)].append(variation)
        
    index_list.append(pd.DataFrame.from_dict(index_dict, orient = "index").T)
    start_idx += 1

11it [04:13, 23.01s/it]


In [19]:
index_df = reduce(lambda  left,right: pd.merge(left,right,on=['date'], how='outer'), index_list)

In [20]:
index_df = index_df.sort_values("date", ascending = False)

In [21]:
index_df.tail()

,date,미국_구리_close,미국_구리_vol,미국_구리_var,영국_구리_close,영국_구리_vol,영국_구리_var,금_close,금_vol,금_var,...,영국_알루미늄_var,은_close,은_vol,은_var,영국_주석_close,영국_주석_vol,영국_주석_var,미국_팔라듐_close,미국_팔라듐_vol,미국_팔라듐_var
2611,2012-01-06,3.4310,0.11K,0.26,7568.75,13.58K,0.53,1616.10,0.02K,-0.20,...,NaN,28.653,0.08K,-2.09,19807.00,0.83K,-0.04,616.10,-,-3.79
2612,2012-01-05,3.4220,0.18K,-0.20,7529.00,13.32K,-0.03,1619.40,0.36K,0.47,...,NaN,29.265,0.03K,0.70,19814.00,1.79K,1.32,640.40,-,-1.46
2613,2012-01-04,3.4290,0.15K,-2.72,7531.25,14.55K,-3.23,1611.90,0.17K,0.76,...,NaN,29.063,0.07K,-1.59,19555.00,1.25K,-1.98,649.90,-,-2.24
2614,2012-01-03,3.5250,0.25K,2.71,7782.25,8.57K,2.47,1599.70,0.23K,2.17,...,NaN,29.533,0.14K,4.88,19949.00,0.47K,4.13,664.80,-,1.42
3173,2012-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,28.160,-,1.02,NaN,NaN,NaN,655.50,-,-0.12


In [22]:
index_df.shape

(3176, 34)

In [23]:
index_df.to_csv(data_path + "metal_df.csv", index = False)